# Updating Allsides Web Scraper
## Get basic AllSides data

In [67]:
import json, requests, pandas as pd
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from copy import deepcopy
from tqdm import tqdm_notebook
#import tqdm from tqdm     <--- for non-Jupyter Notebook

def scrape_page(url, data):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    rows = soup.select('tbody tr')

    for row in rows:
        d = dict()

        d['name'] = row.select_one('.source-title').text.strip()
        d['bias'] = row.select_one('.views-field-field-bias-image a')['href'].split('/')[-1]
        d['allsides_page'] = 'https://www.allsides.com' + row.select_one('.source-title a')['href']
        d['agree'] = int(row.select_one('.agree').text)
        d['disagree'] = int(row.select_one('.disagree').text)
        try:
            ratio = d['agree'] / d['disagree']
            d['agree_ratio'] = ratio
        except ZeroDivisionError:
            d['agree_ratio'] = 1
        d['agreeance_text'] = get_agreeance_text(d['agree_ratio'])

        data.append(d)
     
    # This takes like 90 mins because request limit is 10 secs
    #get_details(data)
    

    
def get_agreeance_text(ratio):
    if ratio > 3: return "absolutely agrees"
    elif 2 < ratio <= 3: return "strongly agrees"
    elif 1.5 < ratio <= 2: return "agrees"
    elif 1 < ratio <= 1.5: return "somewhat agrees"
    elif ratio == 1: return "neutral"
    elif 0.67 < ratio < 1: return "somewhat disagrees"
    elif 0.5 < ratio <= 0.67: return "disagrees"
    elif 0.33 < ratio <= 0.5: return "strongly disagrees"
    elif ratio <= 0.33: return "absolutely disagrees"
    else: return None
    
    
def get_details(data):
    for d in tqdm_notebook(data):
        sleep(10)
        
        r = requests.get(d['allsides_page'])
        soup = BeautifulSoup(r.content, 'html.parser')

        try:
            confidence = soup.select_one('.confidence-level strong').text.strip()
            d['confidence'] = confidence
        except TypeError:
            pass
        try:
            website = soup.select_one('.www')['href']
            d['website_url'] = website
        except TypeError:
            pass
#         try:
#             twitter = soup.select_one('.twitter')['href']
#             d['twitter_url'] = twitter
#         except TypeError:
#             pass
#         try:
#             facebook = soup.select_one('.facebook')['href']
#             d['facebook_url'] = website
#         except TypeError:
#             pass

## Main Script

In [68]:
from copy import deepcopy
from tqdm import tqdm_notebook
from time import sleep
import json


data = []

pages = [
    'https://www.allsides.com/media-bias/media-bias-ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B1%5D=1&field_news_source_type_tid%5B2%5D=2&field_news_bias_nid%5B2%5D=2&field_news_bias_nid%5B3%5D=3&field_news_bias_nid%5B4%5D=4&title=&customFilter=1',
    'https://www.allsides.com/media-bias/media-bias-ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B1%5D=1&field_news_source_type_tid%5B2%5D=2&field_news_bias_nid%5B2%5D=2&field_news_bias_nid%5B3%5D=3&field_news_bias_nid%5B4%5D=4&title=&customFilter=1&page=1',
    'https://www.allsides.com/media-bias/media-bias-ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B1%5D=1&field_news_source_type_tid%5B2%5D=2&field_news_bias_nid%5B2%5D=2&field_news_bias_nid%5B3%5D=3&field_news_bias_nid%5B4%5D=4&title=&customFilter=1&page=2'
    'https://www.allsides.com/media-bias/media-bias-ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B1%5D=1&field_news_source_type_tid%5B2%5D=2&field_news_bias_nid%5B2%5D=2&field_news_bias_nid%5B3%5D=3&field_news_bias_nid%5B4%5D=4&title=&customFilter=1&page=3'
    'https://www.allsides.com/media-bias/media-bias-ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B1%5D=1&field_news_source_type_tid%5B2%5D=2&field_news_bias_nid%5B2%5D=2&field_news_bias_nid%5B3%5D=3&field_news_bias_nid%5B4%5D=4&title=&customFilter=1&page=4'
]

for p in pages:
    scrape_page(p, data)
    sleep(10)
    

with open('AllSidesALL.json', 'w') as f:
    json.dump(data, f)
    
    
#To open it back up:
# with open('allsides.json', 'r') as f:
#     data = json.load(f)